In [35]:
import os
import pandas as pd
import bibtexparser
import numpy as np

curr_dir = os.path.abspath('1.filtrado.ipynb')
bib_db = bibtexparser.parse_file(os.path.join(os.path.dirname(curr_dir), 
                                              'out', 'out.bib'))
data = [v for _,v in {e.key: {a.key: a.value for a in e.fields} for e in bib_db.entries}.items()]

df = pd.DataFrame(data)

print(df.shape)

(417, 55)


In [45]:
req_cols = [
    'title',
    'author',
    'year',
    'abstract',
    'doi',
    'keywords'
]

df_filtered = df.copy()
df_filtered[req_cols] = df[req_cols].replace("", np.nan)
df_filtered.dropna(subset=req_cols, inplace=True)

df_filtered.drop(columns=['pmid'], inplace=True)

print(df_filtered.shape)

(298, 54)


In [46]:
import sqlite3

conn = sqlite3.connect(os.path.join(os.path.dirname(curr_dir), 'db', 
                                    'db.sqlite'))
init_sql_path = os.path.join(os.path.dirname(curr_dir), 'db', 'init.sql')
try:
    with open(init_sql_path, 'r') as f:
        conn.executescript(f.read())
    df_filtered.to_sql('filtered', conn, index=False, if_exists='append')
except Exception as e:
    print(f'Error: {e}')

conn.close()